In [42]:
# default_exp export_named

# Export cells with a name

> Exporting `nbdev` notebook cells with a name adds the ability for a library to differentiate 

In [43]:
from nbdev.export import (is_export, read_nb, find_default_export, 
                          _is_external_export, _re_internal_export, _mk_flag_re,
                         check_re, _re_mod_export, _re_blank_export)
from pathlib import Path
import os
import re
from nbformat import NotebookNode

In [44]:
_re_named_export = _mk_flag_re("exportn_[a-zA-Z0-9:]+", 0,
    "Matches any line with a named #exportn without any module name")

In [45]:
def extract_named_export(cell: NotebookNode):
    name = None
    tst = check_re(cell, _re_named_export)
    if tst:
        pat = re.compile(rf"""exportn_[a-zA-Z0-9:]+""", re.MULTILINE | re.VERBOSE)
        name = pat.search(tst.string).group(0).split('_')[1]
    return name

In [46]:
nb_path = os.path.join(Path('.').resolve().parent, 'examples', 'top2vec.ipynb')
nb = read_nb(nb_path)
named_exports = [ne for ne in [extract_named_export(c) for c in nb['cells']] if ne]
assert(len(named_exports) == 2)

In [102]:
def is_export(cell, default):
    "Check if `cell` is to be exported and returns the name of the module to export it if provided"
    tst = check_re(cell, _re_blank_export)
    if tst:
        if default is None:
            print(f"No export destination, ignored:\n{cell['source']}")
        return default, _is_external_export(tst)
    tst = check_re(cell, _re_mod_export)
    if tst: return os.path.sep.join(tst.groups()[0].split('.')), _is_external_export(tst)
    else: return None

In [119]:
exports = [e for e in [is_export(c, default) for c in nb['cells']]]
assert(exports[0] is None)

In [111]:
def is_export_extended(cell, default):
    "Check if `cell` is to be exported and returns the name of the module to export it if provided"
    tst = check_re(cell, _re_named_export)
    if tst:
        if default is None:
            print(f"No export destination, ignored:\n{cell['source']}")
        return default, _is_external_export(tst)
    tst = check_re(cell, _re_blank_export)
    if tst:
        if default is None:
            print(f"No export destination, ignored:\n{cell['source']}")
        return default, _is_external_export(tst)
    tst = check_re(cell, _re_mod_export)
    if tst: return os.path.sep.join(tst.groups()[0].split('.')), _is_external_export(tst)
    else: return None

In [118]:
exports_extended = [e for e in [is_export_extended(c, default) for c in nb['cells']]]
assert(type(exports_extended[0][0]) == str and exports_extended[0][1])
orig = [e for e in [is_export(c, default) for c in nb['cells']] if e]
new = [e for e in [is_export_extended(c, default) for c in nb['cells']] if e]
assert(len(new) == len(orig)+2)

# Test Notebook2Script & add has named export check

In [47]:
def _notebook2script(fname, modules, silent=False, bare=False):
    "Finds cells starting with `#export` and puts them into a module created by `create_mod_files`"
    bare = str(Config().get('bare', bare)) == 'True'
    if os.environ.get('IN_TEST',0): return  # don't export if running tests
    sep = '\n'* (int(Config().get('cell_spacing', '1'))+1)
    fname = Path(fname)
    nb = read_nb(fname)
    default = find_default_export(nb['cells'])
    if default is not None:
        default = os.path.sep.join(default.split('.'))
    mod = get_nbdev_module()
    exports = [is_export(c, default) for c in nb['cells']]
    cells = [(i,c,e) for i,(c,e) in enumerate(zip(nb['cells'],exports)) if e is not None]
    for i,c,(e,a) in cells:
        if e not in modules: print(f'Warning: Exporting to "{e}.py" but this module is not part of this build')
        fname_out = Config().path("lib_path")/f'{e}.py'
        if bare: orig = "\n"
        else: 
            orig = (f'# {"" if a else "Internal "}C' if e==default else f'# Comes from {fname.name}, c') + 'ell\n'
        flag_lines,code_lines = split_flags_and_code(c)
        code_lines = _deal_import(code_lines, fname_out)
        code = sep + orig + '\n'.join(code_lines)
        names = export_names(code)
        flags = '\n'.join(flag_lines)
        extra,code = extra_add(flags, code)
        code = _from_future_import(fname_out, flags, code, to_dict)
        mod.index.update({f: fname.name for f in names})
        code = re.sub(r' +$', '', code, flags=re.MULTILINE)
        if code != sep + orig[:-1]:
            to_dict[e].append((i, fname, code))
    if not silent: print(f"Converted {fname.name}.")
    return to_dict

In [59]:
from nbdev.export import (Config, get_nbdev_module, 
                          split_flags_and_code, _deal_import,
                          export_names, extra_add, _from_future_import
                         )

In [120]:
def reduced_nb_to_script(fname, to_dict, bare=None):
    bare = str(Config().get('bare', bare)) == 'True'
    sep = '\n'* (int(Config().get('cell_spacing', '1'))+1)
    fname = Path(fname)
    nb = read_nb(fname)
    default = find_default_export(nb['cells'])
    if default is not None:
        default = os.path.sep.join(default.split('.'))
    mod = get_nbdev_module()
    exports = [is_export_extended(c, default) for c in nb['cells']]
    cells = [(i,c,e) for i,(c,e) in enumerate(zip(nb['cells'],exports)) if e is not None]
    for i,c,(e,a) in cells:
        fname_out = Config().path("lib_path")/f'{e}.py'
        if bare: 
            orig = "\n"
        else: 
            named_export = extract_named_export(c)
            if named_export:
                print(f'Found named export {named_export}')
                if e != default:
                    named_export + f' comes from {fname.name}'
                orig = f'# {named_export}'
            else:
                orig = (f'# {"" if a else "Internal "}C' if e==default else f'# Comes from {fname.name}, c') + 'ell\n'
        flag_lines,code_lines = split_flags_and_code(c)
        code_lines = _deal_import(code_lines, fname_out)
        code = sep + orig + '\n'.join(code_lines)
        names = export_names(code)
        flags = '\n'.join(flag_lines)
        extra,code = extra_add(flags, code)
        code = _from_future_import(fname_out, flags, code, to_dict)
        mod.index.update({f: fname.name for f in names})
        code = re.sub(r' +$', '', code, flags=re.MULTILINE)
        if code != sep + orig[:-1]:
            to_dict[e].append((i, fname, code))

In [121]:
fname = str(nb_path)
to_dict = {'top2vec': []}

In [122]:
reduced_nb_to_script(fname, to_dict)

Found named export step:prepproc
Found named export step:preprocess


In [132]:
print(to_dict['top2vec'][0][2])



# step:prepproc
def prepproc():
    print(1)
